# Get triplet

Suppose there is a list of question IDs we are interested in. We want to recover the paragraph related to the question, which is a context, the question, and the ground truth answer. 

In [6]:
# We can load the question id list from json formatted file.
import json
qid_list_filename = "incorrect_ids.json"
with open(qid_list_filename) as f:
    qids = json.load(f)
    
    
print(qids[0:2])
print(len(qids))


['56be4db0acb8001400a502ec', '56be4db0acb8001400a502ed']
10570


In [7]:
# Or we can directly define the list here
qids = ["56be4db0acb8001400a502ec", "56be4db0acb8001400a502ed", "56be4db0acb8001400a502ee"]

print(qids[0:2])
print(len(qids))

['56be4db0acb8001400a502ec', '56be4db0acb8001400a502ed']
3


In [12]:
# Create dictionary for testing membership
qids = dict(zip(qids, [True]*len(qids)))
print(qids)

In [29]:
import os

work_dir = "..\\data\\"
dev_filename = "dev-v1.1.json"

with open(os.path.join(work_dir, dev_filename)) as data_file:    
    dev_data = json.load(data_file)


print("train datafile info:")
print("\tversion:", dev_data["version"])
print()

data = dev_data["data"]


def get_sentence_spans(context):
    from nltk.data import load
    tokenizer = load('tokenizers/punkt/{0}.pickle'.format('english'))
    sentence_span = tokenizer.span_tokenize(context)
    return sentence_span

def get_sentence_index(answer_span, sentence_spans):
    idx = None
    for sent_idx, sent_span in enumerate(sentence_spans):
            if sent_span[0] <= answer_span[0] and answer_span[1] <=sent_span[1]+1:
                idx = sent_idx
                break
    return idx


for article in data:
    title = article["title"]
    # print(title)
    paragraphs = article["paragraphs"]
    for paragraph in paragraphs:
        context = paragraph["context"]
        qas = paragraph["qas"]
        spans = get_sentence_spans(context)
        for qa in qas:
            id = qa["id"]
            if id in qids:
                question = qa["question"]
                answers = qa["answers"]
                
                print("Question ID:", id)
                # print context sentence by sentence
                for idx, sent_span in enumerate(spans):
                    print("%d: %s" %(idx, context[sent_span[0]:sent_span[1]+1]))
                print()
                print("Question:", question)
                print()
                for answer in answers:
                    print("\"%s\" from sentence idx %d" % (answer['text'], get_sentence_index(
                            (answer['answer_start'], answer['answer_start'] + len(answer['text'])),
                            spans
                        )))
                print('---------------------------------------------------------')
                print()
            
    

train datafile info:
	version: 1.1

Question ID: 56be4db0acb8001400a502ec
0: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. 
1: The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. 
2: The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. 
3: As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.

Question: Which NFL team represented the AFC at Super Bowl 50?

"Denver Broncos" from sentence idx 1
"Denver Broncos" from sentence id